# Integrating PydanticAI Tools with the AG2 Framework

In this tutorial, we demonstrate how to integrate [PydanticAI Tools](https://ai.pydantic.dev/tools/) into the AG2 framework. This process enables smooth interoperability between the two systems, allowing developers to leverage PydanticAI's powerful tools within AG2's flexible agent-based architecture. By the end of this guide, you will understand how to configure agents, convert PydanticAI tools for use in AG2, and validate the integration with a practical example.


## Installation
To integrate LangChain tools into the AG2 framework, install the required dependencies:

```bash
pip install ag2[interop-pydantic-ai]
```


In [1]:
import os
from typing import Optional

from pydantic import BaseModel
from pydantic_ai import RunContext
from pydantic_ai.tools import Tool as PydanticAITool

from autogen import AssistantAgent, UserProxyAgent
from autogen.interop.pydantic_ai import PydanticAIInteroperability

## Agent Configuration

Configure the agents for the interaction.
- `config_list` defines the LLM configurations, including the model and API key.
- `UserProxyAgent` simulates user inputs without requiring actual human interaction (set to `NEVER`).
- `AssistantAgent` represents the AI agent, configured with the LLM settings.

In [2]:
config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
)

chatbot = AssistantAgent(
    name="chatbot",
    llm_config={"config_list": config_list},
)

In [3]:
class Player(BaseModel):
    name: str
    age: int


def get_player(ctx: RunContext[Player], additional_info: Optional[str] = None) -> str:  # type: ignore[valid-type]
    """Get the player's name.

    Args:
        additional_info: Additional information which can be used.
    """
    return f"Name: {ctx.deps.name}, Age: {ctx.deps.age}, Additional info: {additional_info}"  # type: ignore[attr-defined]


pydantic_ai_interop = PydanticAIInteroperability()
pydantic_ai_tool = PydanticAITool(get_player, takes_ctx=True)

player = Player(name="Luka", age=25)
ag2_tool = pydantic_ai_interop.convert_tool(tool=pydantic_ai_tool, deps=player)

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

In [ ]:
user_proxy.initiate_chat(
    recipient=chatbot, message="Get player, for additional information use 'goal keeper'", max_turns=3
)